# Data Cleaning and Preparation

In [9]:
import numpy as np
import pandas as pd


## Data Transformation (continue)
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

### Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value
replacement. As you’ve already seen, `map` can be used to modify a subset of values in
an object but `replace` provides a simpler and more flexible way to do so. 

Let’s consider this Series

In [91]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [24]:
# replace the value of -999 with np.nan


In [25]:
# do the same with map function


If you want to replace multiple values at once, you instead pass a list and then the
substitute value

In [26]:
# replace both the values of -999 and -1000 with np.nan


To use a different replacement for each value, pass a list of substitutes, or a mapping dict 


In [27]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly


In [28]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly


### Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping 
of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure.

In [111]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method

In [29]:
# convert the indices to upper case


If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`

In [30]:
# convert the indices and column names to upper case using rename method


In [31]:
# rename spcific index and column


In [32]:
# rename index in-place


## Independent Practice
using the `coffee-preferences.csv` data set
- show the duplicated rows
- show the names of customers who rated all coffee
- show the names of the coffees that are rated by all customers
- which coffee got the highest rate in average
- replace the missing value by zero
- convert names to upper case
- convert the rate to a percentage 

### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets

In [135]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use `cut`, a function in pandas

In [1]:
# divide the ages to 4 classes using the bins
bins = [18, 25, 35, 60, 100]


The object pandas returns is a special **Categorical object**. The output you see
describes the bins computed by `pandas.cut`. 

You can treat it like an **array of strings** indicating the bin name; 

internally it contains a **categories array** specifying the distinct category names along with a labeling for the ages data in the **codes attribute**:

In [2]:
# print the ages codes


In [3]:
# print the distinct categories


In [4]:
# print the number of occurrances of each category 


**Note** that `cats.value_counts()` are the bin counts for the result of `pandas.cut`.
Consistent with mathematical notation for intervals, a **parenthesis** means that the side
is **open**, while the **square bracket** means it is **closed (inclusive)**. 

You can change which side is closed by passing `right=False`:

In [5]:
# categorize the ages with right open intervals


You can also pass your own bin names by passing a list or array to the labels option:

In [6]:
# categorize the ages with the labels below
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']


If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

In [142]:
data = np.random.randint(0, 20, 12)
data

array([15, 13, 13, 12, 15,  2, 13, 10, 10,  4, 17,  9])

In [7]:
# divide the data to 4 groups


In [8]:
# count the number of values in each group


A closely related function, `qcut`, bins the data based on sample quantiles. 

Depending on the distribution of the data, using `cut` will not usually result in each bin having the
same number of data points. 

Since `qcut` uses sample quantiles instead, by definition you will obtain **roughly equal-size** bins:

In [10]:
data = np.random.randint(0, 100, 1000)  # Normally distributed
# Cut into quartiles and count the values


### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations.
Consider a DataFrame with some normally distributed data

In [45]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.008590,0.017220,-0.021223,0.035455
std,1.025077,0.984128,0.994651,1.030732
min,-2.653837,-3.276326,-2.794111,-3.070852
25%,-0.747240,-0.643613,-0.685595,-0.615122
50%,0.002157,0.016367,-0.050036,0.041291
75%,0.626514,0.694616,0.616174,0.691573
max,3.538239,3.020526,3.107429,3.927963


In [47]:
# find values in one of the columns exceeding 3 in absolute value


In [48]:
# select all rows having a value exceeding 3 or –3
mask = (np.abs(data) > 3).any(axis=1)
data[mask]

,0,1,2,3
323,0.819170,0.205394,3.107429,2.389105
565,-0.180680,2.098918,0.389456,3.085096
631,0.141573,-0.595428,0.478071,3.742111
671,2.265580,1.210193,-0.013171,3.927963
687,3.391274,-0.542325,0.327023,-2.130023
850,0.367752,3.020526,0.281012,-0.183666
874,3.085712,0.844944,0.435273,-2.566219
884,0.112020,-0.406228,0.579325,-3.070852
904,3.538239,0.736581,0.130792,0.158842
933,1.271074,-3.276326,1.739162,0.167909


In [58]:
# set outliers to 3 or -3 depending on its sign


### Computing Indicator/Dummy Variables
Another type of transformation for statistical modeling or machine learning applica‐
tions is converting a categorical variable into a “dummy” or “indicator” matrix.

column in a DataFrame has k distinct values, you would derive a matrix or Data‐
Frame with k columns containing all 1s and 0s.

pandas has a `get_dummies` function for doing this

In [9]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [41]:
# create dummy variables for column 'key'


In [42]:
# create dummy variables for column 'key'  then add the columns to a variable


In [43]:
# join the dummies to the Data-Frame
